In [4]:
import argparse
import sys
import pickle
import math
import os

import torch
from transformers import LayoutLMv3Tokenizer, AutoConfig

sys.path.append('../src')
from model import My_DataLoader
from model.LayoutLMv3forMIM import LayoutLMv3ForPretraining
from utils import masking_generator, utils

In [8]:
parser = argparse.ArgumentParser()
parser.add_argument("--tokenizer_vocab_dir", type=str, required=True)
parser.add_argument("--input_file", type=str, required=True)
parser.add_argument("--model_params", type=str)
parser.add_argument("--ratio_train", type=float,default=0.9)
parser.add_argument("--output_model_dir", type=str, required=True)
parser.add_argument("--output_file_name", type=str, required=True)
parser.add_argument("--model_name", type=str, required=True)
parser.add_argument("--batch_size", type=int, default=2)
parser.add_argument("--leaning_rate", type=int, default=1e-5)
parser.add_argument("--max_epochs", type=int, default=1)
args_list = ["--tokenizer_vocab_dir", "../data/vocab/tokenizer_vocab/","--input_file",
            "../data/preprocessing_shared/30000data_10000_split/",
            "--output_model_dir", "../data/train/model/", \
            "--output_file_name", "model.param", \
            "--batch_size", "4", \
            "--model_name", "microsoft/layoutlmv3-base"]
args = parser.parse_args(args_list)

In [9]:
tokenizer = LayoutLMv3Tokenizer(f"{args.tokenizer_vocab_dir}vocab.json", f"{args.tokenizer_vocab_dir}merges.txt")
ids = range(tokenizer.vocab_size)
vocab = tokenizer.convert_ids_to_tokens(ids)

In [10]:
visual_bbox = utils.init_visual_bbox()

In [11]:
visual_bbox.shape

torch.Size([196, 4])

In [13]:
with open(args.input_file + "0.pkl", 'rb') as f:
    data = pickle.load(f) 

In [ ]:
len(data)

23274

In [ ]:
data = []
input_names = os.listdir(args.input_file)
for file_name in input_names:
    print(file_name)
    with open(f"{args.input_file}{file_name}", "rb") as f:
        d = pickle.load(f)
        data += d

0.pkl
10000.pkl
20000.pkl


In [14]:
len(data)

23274

In [20]:
[] + [1, 3 ] + [4, 6]

[1, 3, 4, 6]

In [15]:
if  args.model_params is not None:
    model = torch.load(args.model_params)
else:
    config = AutoConfig.from_pretrained(args.model_name)
    config.num_visual_tokens = 8192
    model = LayoutLMv3ForPretraining(config)
    # Roberta_model = RobertaModel.from_pretrained("roberta-base")
    # ## embedidng 層の重みをRobertaの重みで初期化
    # weight_size = model.state_dict()["model.embeddings.word_embeddings.weight"].shape
    # for i in range(weight_size[0]):
    #   model.state_dict()["model.embeddings.word_embeddings.weight"][i] = \
    #   Roberta_model.state_dict()["embeddings.word_embeddings.weight"][i]

In [16]:
n_train = math.floor(len(data) * args.ratio_train)
train_data = data[:n_train]
valid_data = data[n_train:]
print(len(train_data), len(valid_data))

my_dataloader = My_DataLoader.My_Dataloader(vocab)
train_dataloader = my_dataloader(train_data, batch_size=args.batch_size, shuffle=True)
valid_dataloader = my_dataloader(valid_data, batch_size=args.batch_size, shuffle=False)

20946 2328


In [17]:
losses = []
model.train()
for epoch in range(args.max_epochs):
    for iter, batch in enumerate(train_dataloader):
        # inputs = {k: v.to(f'cuda:{model.device_ids[0]}') for k in ["input_ids, bbox", "pixel_values", "attention_mask"]}
        inputs = {k: batch[k] for k in ["input_ids", "bbox", "pixel_values", "attention_mask", "bool_mi_pos"]}
        text_logits, image_logits, wpa_logits = model.forward(inputs)
        break
x

torch.Size([4, 196, 768])
torch.Size([1, 1, 768])
torch.Size([4, 196, 1])


/home/is/hikaru-si/.pyenv/versions/3.8.6/envs/exp_005/lib/python3.8/site-packages/transformers/modeling_utils.py:713: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


NameError: name 'x' is not defined

In [ ]:
batch["alignment_label"].shape, batch["attention_mask"].shape

(torch.Size([4, 512]), torch.Size([4, 512]))

(tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1.,

In [8]:
wpa_logits.

SyntaxError: invalid syntax (3386499572.py, line 1)

In [9]:
def _init_visual_bbox(img_size=(14, 14), max_len=1000):
    #torch div : divide
    visual_bbox_x = torch.div(torch.arange(0, max_len * (img_size[1] + 1), max_len),
                              img_size[1], rounding_mode='trunc')
    visual_bbox_y = torch.div(torch.arange(0, max_len * (img_size[0] + 1), max_len),
                              img_size[0], rounding_mode='trunc')
    visual_bbox = torch.stack(
        [
            visual_bbox_x[:-1].repeat(img_size[0], 1),
            visual_bbox_y[:-1].repeat(img_size[1], 1).transpose(0, 1),
            visual_bbox_x[1:].repeat(img_size[0], 1),
            visual_bbox_y[1:].repeat(img_size[1], 1).transpose(0, 1),
        ],
        dim=-1,
    ).view(-1, 4)
    return visual_bbox

In [10]:
visual_bbox = _init_visual_bbox()
visual_bbox.shape

torch.Size([196, 4])

In [11]:
# (x0, y0, x1, y1) x0, y0比較
def is_content_bbox(text_bbox, image_bbox):
    if (text_bbox[0] >= image_bbox[0] and text_bbox[1] >= image_bbox[1] 
      and text_bbox[0] <= image_bbox[2] and text_bbox[1] <= image_bbox[3]):
        return True
    else:
        return False

In [12]:
# (x0, y0, x1, y1) x1, y1比較
def is_content_bbox_2(text_bbox, image_bbox):
    if (text_bbox[2] >= image_bbox[0] and text_bbox[3] >= image_bbox[1] 
      and text_bbox[2] <= image_bbox[2] and text_bbox[3] <= image_bbox[3]):
        return True
    else:
        return False

In [9]:
def aligment(text_bbox):
    alignment_list = []
    for i in range(len(text_bbox)):
        alignment = []
        for j in range(len(visual_bbox)):
            if is_content_bbox(text_bbox[i], visual_bbox[j]):
                alignment.append(j)
            if is_content_bbox_2(text_bbox[i], visual_bbox[j]):
                alignment.append(j)
        alignment_list.append(alignment)

    return alignment_list

In [18]:
def create_alignment_label(visual_bbox, text_bbox, bool_mi_pos):
    num_text = len(text_bbox)
    labels = torch.ones(num_text)
    for v_b in visual_bbox[bool_mi_pos]:
        for j, t_b in enumerate(text_bbox):
            if is_content_bbox(t_b, v_b) or is_content_bbox_2(t_b, v_b):
                labels[j] = 0
    alignment_label = labels.to(torch.bool)
    return alignment_label

In [18]:
al_label = utils.create_alignment_label(visual_bbox, batch["bbox"][0], batch["bool_mi_pos"][0])

In [30]:
a = batch["bool_mi_pos"][0].unsqueeze(0)
a.shape

torch.Size([1, 196])

In [31]:
a[0].shape

torch.Size([196])

In [13]:
bool_mi_pos[0][[44, 45]], is_content[132], bool_mi_pos[0].sum()

NameError: name 'bool_mi_pos' is not defined

In [14]:
wpa_alignment = []
for indexes in is_content:
    if bool_mi_pos[0][indexes].sum().item() == 0:
      label = 1
    else:
      label = 0
    wpa_alignment.append(label)

NameError: name 'is_content' is not defined

In [11]:
## fast!
l = []
for v_b in visual_bbox[bool_mi_pos[2]]:
    for i, t_b in enumerate(text_bbox):
        if is_content_bbox(t_b, v_b) or is_content_bbox_2(t_b,v_b):
            l.append(i)

NameError: name 'visual_bbox' is not defined

In [12]:
bool_mi_pos.shape

NameError: name 'bool_mi_pos' is not defined

In [13]:
torch.ones(512).shape

torch.Size([512])

In [14]:
#対応する画像がmaskされている 0 False, maskされていない: 1 True
def crete_alignment_label(visual_bbox, text_bboxes, bool_mi_pos):
    num_batch, num_text, _ = text_bboxes.shape
    if num_batch != bool_mi_pos.shape[0]:
        print("difarent batch size!")
    alignment_labels = []
    for i in range(num_batch):
        labels = torch.ones(num_text)
        for v_b in visual_bbox[bool_mi_pos[i]]:
            for j, t_b in enumerate(text_bboxes[i]):
                if is_content_bbox(t_b, v_b) or is_content_bbox_2(t_b, v_b):
                    labels[j] = 0
        alignment_labels.append(labels.to(torch.bool))
    return torch.stack(alignment_labels)

In [21]:
la = crete_alignment_label(visual_bbox, batch["bbox"], batch["bool_mi_pos"])

In [23]:
la.shape

torch.Size([4, 512])

In [258]:
seq_len = batch["attention_mask"][0].sum().item()
print(int(seq_len))

243


In [263]:
wpa_label = la[0][:int(seq_len)]
wpa_label.shape

torch.Size([243])

In [266]:
wpa_label[][""] 

torch.Size([243])

In [265]:
batch["ml_position"][0]

tensor([  9,  10,  23,  24,  25,  26,  27,  28,  34,  35,  36,  44,  45,  46,
         76,  77,  78,  79,  80,  91,  92, 105, 106, 107, 108, 115, 116, 117,
        118, 119, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 134,
        135, 136, 137, 138, 139, 140, 141, 142, 182, 191, 192, 193, 194, 220,
        221, 222, 223, 224, 225, 226, 227, 228, 229, 230])

In [248]:
la[0][batch["attention_mask"][0].to(torch.bool)].

tensor(183)

In [215]:
t_b = [0, 0, 0, 0]

In [220]:
0 == sum(t_b)

True

In [234]:
batch["bbox"][0][la[0]]

torch.Size([452, 4])

In [205]:
a = torch.ones(10)
b = torch.zeros(10)

In [207]:
torch.stack([a, b]).shape

torch.Size([2, 10])

In [181]:
len(l),text_bbox[l]

(3,
 tensor([[514, 686, 538, 700],
         [544, 686, 608, 700],
         [514, 702, 569, 716]]))

In [182]:
b = visual_bbox[bool_mi_pos[0]][0]
b

tensor([214,   0, 285,  71])

In [183]:
mask_index = torch.arange(0, 196)[bool_mi_pos[0]]

In [32]:
batch["bool_mi_pos"][1].shape, batch["bbox"][0].shape

(torch.Size([196]), torch.Size([512, 4]))

In [37]:
visual_bbox.shape, batch["bool_mi_pos"][0].shape

(torch.Size([196, 4]), torch.Size([196]))

In [28]:
al_label = create_alignment_label(visual_bbox, batch["bbox"][1], batch["bool_mi_pos"][1])

In [26]:
from PIL import Image, ImageDraw

In [29]:
rect = Image.new("RGBA", (1000, 1000), (0, 0, 255))
rect_d = ImageDraw.Draw(rect)

for b in batch["bbox"][1].tolist():
  rect_d.rectangle(b, outline=(0, 255, 0))
for b in batch["bbox"][1][al_label].tolist():
  rect_d.rectangle(b, fill=(0, 225, 0), outline=(0, 255, 225))
for b in visual_bbox[batch["bool_mi_pos"][1]].tolist():
  rect_d.rectangle(b, outline=(225, 0, 0))

rect.show()
rect.save("mask.png")

Error: no "view" mailcap rules found for type "image/png"


In [241]:
batch["bbox"][0].shape, la.shape

(torch.Size([512, 4]), torch.Size([4, 512]))

In [120]:
# from PIL import Image, ImageDraw

# rect = Image.new("RGBA", (1000, 1000), (0, 0, 255))

# rect_d = ImageDraw.Draw(rect)
# rect_d.rectangle(
#     [0, 0, 500, 300], fill=(255, 0, 0)
# )

# rect.show()
# rect.save("mask.png")

Error: no "view" mailcap rules found for type "image/png"


In [43]:
window_size = (14, 14)
num_masking_patches = 75
max_mask_patches_per_block = None
min_mask_patches_per_block = 16

In [44]:
mask_generator = masking_generator.MaskingGenerator(
            window_size, num_masking_patches=num_masking_patches,
            max_num_patches=max_mask_patches_per_block,
            min_num_patches=min_mask_patches_per_block,
        )

In [49]:
bool_masked_pos = mask_generator()
bool_masked_pos = torch.from_numpy(bool_masked_pos).unsqueeze(0)
bool_masked_pos = bool_masked_pos.flatten(1).to(torch.bool)

In [51]:
bool_masked_pos[0].shape

torch.Size([196])

In [52]:
batch["bool_mi_pos"].shape

torch.Size([4, 196])

In [58]:
a = torch.ones([1, 196])

In [59]:
torch.cat([a, a]).shape

torch.Size([2, 196])